In [1]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.storage import get_availability_model_data, get_demand_model_data
from modules.config import *
from modules.svm import *

# Support Vector Machine (SVM) Training
In this notebook we train SVM to predict availability. This process is split into two stages. Firstly, we determine the best hyperparameters for a fixed H3 and time resolution. Then, we use these hyperparameters to train models for all resolutions.

In the first stage, we perform three separate grid searches for 3 kernels - linear, rbf, polynomial. In order to speed up the training we use a grid search with successive halving, which first evaluates all candidates (combinations of hyperparameters) with a small fraction of the training set and then trains the best third of all candidates on a fraction of the dataset that is three times as large as in the previous stage. This process repeats until the best hyperparameters are found and evaluated on the whole dataset. By using halving grid search we hope to speed up the first stage.

As it states in scikit-learn documentation:

"The implementation is based on libsvm. The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to datasets with more than a couple of 10000 samples."

We have attempted model fitting on training sets of different size and found 30000 to be the optimal to achieve comparable results to other models. The execution time already lasted between 6 and 10 hours depending on current RAM usage, which is why we do not advise to run this notebook.


For all three kernels we use various values on a log scale for the regularization parameter C, which tries to minimize the coefficients of the model. For the RBF kernel we also use different values on a log scale for the bandwidth γ, which determines how smooth the feature functions are. Lastly, we vary
the degree of the polynomial d for the polynomial kernel.

In [2]:
all_possible_metas = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
]

In [3]:
execute_stage(
    'first_stage',
    SVM_FIRST_STAGE_AVAILABILITY_RESULTS_PATH,
    SVM_SECOND_STAGE_AVAILABILITY_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    all_possible_metas,
    get_availability_model_data,
)

h3_res: 8 - time_interval_length: 6 - param_kernel: linear - param_C: 0.1 - param_gamma: -1 - param_degree: -1 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: linear - param_C: 1 - param_gamma: -1 - param_degree: -1 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: linear - param_C: 10 - param_gamma: -1 - param_degree: -1 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: linear - param_C: 100 - param_gamma: -1 - param_degree: -1 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: rbf - param_C: 0.1 - param_gamma: 0.1 - param_degree: -1 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: rbf - param_C: 0.1 - param_gamma: 0.01 - param_degree: -1 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: rbf - param_C: 0.1 - param_gamma: 0.001 - param_degree: -1 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: rbf - param_C: 0.1 - param_gamma: 0.0001 - param_de

0it [00:00, ?it/s]

In [4]:
results = pd.read_parquet(SVM_FIRST_STAGE_AVAILABILITY_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(2)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
53,2,29997,581.399808,5.128612,45.074160,2.983482,100.0,poly,1000000,"{'C': 100.0, 'degree': 3.0, 'gamma': None, 'ke...",...,-1.835908,-1.801097,-1.700345,-1.824533,0.073370,0,8,6,NaN,3.0
28,2,29997,388.042569,1.615363,49.679345,0.525620,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-3.208867,-3.032053,-3.078978,-3.103936,0.059588,0,8,6,0.01,NaN


In [5]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel')[[
    'mean_test_score',
    'param_C',
    'param_degree',
    'param_gamma',
]].first()

,mean_test_score,param_C,param_degree,param_gamma
param_kernel,,,,
linear,-9.517249,10.0,NaN,NaN
poly,-4.953725,100.0,3.0,NaN
rbf,-5.178454,10.0,NaN,0.01


As we can see the best performing model is with polynomial kernel, degree 3 and regularization parameter C equal 100. It is closely followed by the model with rbf kernel.

In [6]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    40.411474
poly      34.547975
rbf       23.520531
Name: mean_fit_time, dtype: float64

Next, we train models for all resolutions with found hyperparameters.

In [7]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [8]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        'second_stage',
        SVM_FIRST_STAGE_AVAILABILITY_RESULTS_PATH,
        SVM_SECOND_STAGE_AVAILABILITY_RESULTS_PATH,
        h3_res,
        time_interval_length,
        all_possible_metas,
        get_availability_model_data,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

h3_res: 7 - time_interval_length: 1 - param_kernel: poly - param_C: 100.0 - param_gamma: nan - param_degree: 3.0 # already trained
h3_res: 7 - time_interval_length: 2 - param_kernel: poly - param_C: 100.0 - param_gamma: nan - param_degree: 3.0 # already trained
h3_res: 7 - time_interval_length: 6 - param_kernel: poly - param_C: 100.0 - param_gamma: nan - param_degree: 3.0 # already trained
h3_res: 7 - time_interval_length: 24 - param_kernel: poly - param_C: 100.0 - param_gamma: nan - param_degree: 3.0 # already trained
h3_res: 8 - time_interval_length: 1 - param_kernel: poly - param_C: 100.0 - param_gamma: nan - param_degree: 3.0 # already trained
h3_res: 8 - time_interval_length: 2 - param_kernel: poly - param_C: 100.0 - param_gamma: nan - param_degree: 3.0 # already trained
h3_res: 8 - time_interval_length: 6 - param_kernel: poly - param_C: 100.0 - param_gamma: nan - param_degree: 3.0 # already trained
h3_res: 8 - time_interval_length: 24 - param_kernel: poly - param_C: 100.0 - param

In [9]:
results = pd.read_parquet(SVM_SECOND_STAGE_AVAILABILITY_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_kernel,param_max_iter,...,std_train_score,n_iter,h3_res,time_interval_length,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy,param_gamma
8,0,30000,514.563769,2.812394,103.005932,2.033204,100.0,3.0,poly,10000000,...,0.020691,384813,9,24,1.914686,1.383722,0.712426,0.627857,0.809592,NaN
5,0,30000,263.425706,2.414496,42.652927,3.130323,100.0,3.0,poly,10000000,...,0.016788,370011,8,2,4.545798,2.132088,0.978162,0.504630,0.832024,NaN
6,0,30000,267.706322,3.910984,38.311128,2.404458,100.0,3.0,poly,10000000,...,0.033305,495873,8,6,4.928011,2.219912,0.989950,0.475697,0.850278,NaN
4,0,30000,264.819012,5.537926,45.179344,4.544869,100.0,3.0,poly,10000000,...,0.023578,269945,8,1,4.681280,2.163626,1.029570,0.527839,0.809220,NaN
7,0,30000,243.093547,3.142554,37.623025,3.557415,100.0,3.0,poly,10000000,...,0.068687,390633,8,24,6.854364,2.618084,1.181139,0.404322,0.878042,NaN
1,0,30000,1775.216243,40.427614,9.656882,1.164042,100.0,3.0,poly,10000000,...,0.508076,10000000,7,2,35.567582,5.963856,2.545455,0.355072,0.733247,NaN
0,0,30000,353.543360,11.154014,16.253207,4.090306,100.0,3.0,poly,10000000,...,0.277427,2011781,7,1,37.127169,6.093207,2.948520,0.386876,0.557705,NaN
2,0,30000,2130.720859,10.375939,11.490340,2.429825,100.0,3.0,poly,10000000,...,0.163310,10000000,7,6,35.610647,5.967466,2.313836,0.307780,0.889143,NaN
3,0,9855,28.980687,1.775780,1.144874,0.239486,100.0,3.0,poly,10000000,...,0.803591,618653,7,24,55.988396,7.482539,3.165796,0.256407,0.768442,NaN
